# Find clusters of politicians spending with companies owned by each others relatives or himself (issue #18)

In this analysis we try to identify the pattern described bellow:

* According to "ATO DA MESA Nº 43, DE 21/5/2009 Art. 4º § 13", no expense can be reimbursed if made with companies where the applicant (politician) is owner, partner or relative (maximum of third degree).

Politicians may know about this and have groups to "exchange expenses" to cheat the law. e.g. Politician A pays for Politician B's relatives; Politician B pays for Politician A's relatives.

In [1]:
from unicodedata import normalize
def normalize_name(name):
    return normalize('NFKD', name).encode('ASCII', 'ignore').decode('utf-8').upper()

def normalize_list(my_list):
    result = []
    for n in my_list:
        result.append(normalize_name(n))
    return result

import requests
import json
import os
import time

cache = {}
cache_path = '../data/cnpj_cache.json'
if os.path.exists(cache_path):
    cache = json.loads(open(cache_path, 'r').read())

def save_cnpj_cache():
    global cache
    open(cache_path, 'w').write(json.dumps(cache))
    
def extract_cnpj_board(cnpj):
    global cache
    result = []
    try:
        if cnpj not in cache:
            url = 'http://receitaws.com.br/v1/cnpj/%s' % cnpj
            try:
                text = requests.get(url, timeout=5).text
                if 'too many' in text.lower():
                    time.sleep(1)
                    return extract_cnpj_board(cnpj)
                data = json.loads(text)

                cache[cnpj] = data
            except:
                cache[cnpj] = {'error': 'timeout'}
        else:
            data = cache[cnpj]

        if 'qsa' in data:
            for info in data['qsa']:
                result.append(normalize_name(info['nome']))
    except:
        #print('Error collecting data from:', cnpj)
        pass
    return result

Load database of reimbursements to start our analysis

In [2]:
import pandas as pd
import numpy as np
import json

df = pd.read_csv('../data/2016-12-07-reimbursements.xz',
                 dtype={ 'cnpj_cpf': np.str, 'reimbursement_numbers': np.str })

First lets simulate the situation described in the news
* Situation: http://m.congressoemfoco.uol.com.br/noticias/deputado-gasta-r-189-mil-da-cota-para-abastecer-no-posto-do-irmao/
* Comment about this process: https://github.com/datasciencebr/serenata-de-amor/issues/17#issuecomment-254656648


So as described in the issue I'll do the same following steps:

* I did need to find the fullname of deputy "Wellington Roberto" and achieve this in: http://www.camara.leg.br/internet/deputado/dep_Detalhe.asp?id=5830395
* So I would like to find out all of companies that him is involved as partner:
http://www.consultasocio.com/q/sa/jose-wellington-roberto
* There I found these names: Jose Edvan Roberto, Milena Roberto De Assis, Alto Branco Participacoes S/S Ltda, Walkiria Henriques Roberto(That I judge that all partner are part of family of deputy Jose Wellington Roberto).
So I did research for "Jose Edvan Roberto" and bazinga!!!!
http://www.consultasocio.com/q/sa/jose-edvan-roberto

* This guy(brother of wellington roberto) owns "Posto de Combustivel Shopping Ltda" company with all Roberto's family.

In [3]:
# Step 1: Collect the civil name of Wellington Roberto

# Collect and merge with the main DataFrame the Civil Names 
# (the database used is generated by src/fetch_civil_names.py and of course it'll take a while)
civil_names = pd.read_csv('../data/2016-12-07-congressperson-civil-names.xz')
df = pd.merge(left=df,right=civil_names, left_on='congressperson_id', right_on='congressperson_id', how='right')

# Step 2: Get all business partners
# Let's open the Business Partners database that you can download here: (link)
bp_database = json.loads(open('../data/2016-12-13-congressperson-business-partners.json', 'r').read())

If any company is owned by the people showed above it's illegal and we have a problem because they are business partners.

In [78]:
# Step 3: Look for expenses that are made in companies owned by these people

def identify_fraud(congressperson_name):
    if len(df[df['congressperson_name'] == congressperson_name]):
        civil_name = normalize_name(df[df['congressperson_name'] == congressperson_name]['civil_name'].iloc[0])
        if civil_name in bp_database:
            business_partners = bp_database[civil_name]['partners']

            for cnpj in df[df['congressperson_name'] == congressperson_name]['cnpj_cpf'].unique():
                board_names = extract_cnpj_board(cnpj)
                if len(set(normalize_list(business_partners)) & set(normalize_list(board_names))):
                    print('- Possibly fraud detected', cnpj)
        else:
            print('WARN: %s not found in business partners database' % civil_name)

In [80]:
id = 0
total = len(df['congressperson_name'].unique())
for name in df['congressperson_name'].unique():
    id += 1
    print('%d/%d: %s' % (id, total, name))
    identify_fraud(name)
    save_cnpj_cache()

1/1089: DILCEU SPERAFICO
2/1089: DOMINGOS DUTRA
3/1089: EDINHO BEZ
4/1089: EDINHO ARAÚJO
5/1089: EDSON EZEQUIEL
6/1089: EDUARDO BARBOSA
7/1089: ELCIONE BARBALHO
8/1089: ELISEU PADILHA
9/1089: ENIO BACCI
10/1089: EURIPEDES MIRANDA
11/1089: FERNANDO FERRO
12/1089: FERNANDO GABEIRA
13/1089: GASTÃO VIEIRA
14/1089: HERMES PARCIANELLO
15/1089: IVAN VALENTE
16/1089: JAIME MARTINS
17/1089: JOÃO PIZZOLATTI
18/1089: JOÃO MATOS
19/1089: JOÃO LEÃO
20/1089: JOÃO PAULO CUNHA
21/1089: MENDES RIBEIRO FILHO
22/1089: INOCÊNCIO OLIVEIRA
23/1089: JOSÉ ROCHA
24/1089: ARNON BEZERRA
25/1089: JOSÉ PIMENTEL
26/1089: JOSÉ PRIANTE
27/1089: ZÉ GERARDO
28/1089: MENDONÇA FILHO
29/1089: SARAIVA FELIPE
30/1089: JOSÉ CHAVES
31/1089: JOVAIR ARANTES
32/1089: JÚLIO CESAR
33/1089: MARCIO REINALDO MOREIRA
34/1089: MARINHA RAUPP
35/1089: MAURO LOPES
36/1089: NELSON MEURER
37/1089: NEWTON CARDOSO
38/1089: ODÍLIO BALBINOTTI
39/1089: PAULO BORNHAUSEN
40/1089: PEDRO WILSON
41/1089: RÉGIS DE OLIVEIRA
42/1089: ROBERTO ROCHA
43/10

In the report above we found some possibly frauds, let's check all of them:

55/1089: CIRO NOGUEIRA - CNPJ: 01971900000149

175/1089: MARIA HELENA - CNPJ: 15522508000127

210/1089: SANDRA ROSADO - CNPJ: 08314429000165

219/1089: WELLINGTON ROBERTO - CNPJ: 09307232000161

294/1089: FELIPE MAIA - CNPJ: 07083712000160

338/1089: LUIZ FERNANDO FARIA - CNPJ: 24574923000115

408/1089: VITOR PENIDO - CNPJ: 00152489000117

501/1089: JORGE CÔRTE REAL - CNPJ: 08872798000173

566/1089: BERINHO BANTIM - CNPJ: 14331631000106

592/1089: JOSÉ NUNES - CNPJ: 08008566000171

670/1089: FERNANDO JORDÃO - CNPJ: 02409350000131 and 03825735000142

758/1089: LUCIANO CASTRO - CNPJ: 01271789000188

770/1089: ARMANDO ABÍLIO - CNPJ: 12610937000168

773/1089: BETINHO ROSADO - CNPJ: 12997664000156 and 24186785000105

898/1089: RAQUEL MUNIZ - CNPJ: 16902140000195

934/1089: MARINALDO ROSENDO - CNPJ: 10651703000133

971/1089: FÁBIO MITIDIERI - CNPJ: 00826182000154

977/1089: RODRIGO PACHECO - CNPJ: 17272550000162

1034/1089: LUCIO MOSQUINI - CNPJ: 11601349000103

In [82]:
def show_fraud_info(congressperson_name, cnpj):
    civil_name = normalize_name(df[df['congressperson_name'] == congressperson_name]['civil_name'].iloc[0])
    board_names = extract_cnpj_board(cnpj)
    
    business_partners = bp_database[civil_name]['partners']
    frauds = df.loc[(df['congressperson_name'] == congressperson_name) & (df['cnpj_cpf'] == cnpj), 
                    ['issue_date', 'cnpj_cpf', 'supplier', 'total_net_value']]
    print('Congress Name: %s' % congressperson_name)
    print('Civil Name: %s' % civil_name)
    print('CNPJ Business Partners Name:', list(set(normalize_list(business_partners)) & set(normalize_list(board_names))))
    print('%d suspect transactions in amount of R$ %.2f' % (len(frauds.total_net_value), frauds.total_net_value.sum()))
    return frauds

# Nepotism detected

## Sandra Maria da Escóssia Rosado

ATTENTION: In the case bellow we have a fraud!

Because SANDRA MARIA DA ESCÓSSIA ROSADO is wife of LAIRE ROSADO FILHO

More information about the family: http://www.blogdobg.com/tag/sandra/

In [83]:
show_fraud_info('SANDRA ROSADO', '08314429000165')

Congress Name: SANDRA ROSADO
Civil Name: SANDRA MARIA DA ESCOSSIA ROSADO
CNPJ Business Partners Name: ['LAIRE ROSADO FILHO', 'JERONIMO VINGT ROSADO MAIA']
7 suspect transactions in amount of R$ 44000.00


,issue_date,cnpj_cpf,supplier,total_net_value
215378,2011-02-07T00:00:00,08314429000165,EDITORA DE JORNAIS LTDA.,5000.0
215432,2011-03-02T00:00:00,08314429000165,EDITORA DE JORNAIS LTDA,5000.0
215462,2011-03-22T00:00:00,08314429000165,REDE RESISTÊNCIA DE COMUNICAÇÃO,5000.0
215505,2011-04-25T00:00:00,08314429000165,EDITORA DE JORNAIS LTDA.,5000.0
215606,2011-05-24T00:00:00,08314429000165,EDITORA DE JORNAIS LTDA,8000.0
215688,2011-06-21T00:00:00,08314429000165,REDE RESISTÊNCIA DE COMUNICAÇÃO,8000.0
215786,2011-08-02T00:00:00,08314429000165,REDE RESISTÊNCIA DE COMUNICAÇÃO,8000.0


## José Wellington Roberto

ATTENTION: In the case bellow we have a fraud! 

Because José Wellington Roberto is brother of josé Edvan Roberto and it's not allowed!

In [84]:
show_fraud_info('WELLINGTON ROBERTO', '09307232000161')

Congress Name: WELLINGTON ROBERTO
Civil Name: JOSE WELLINGTON ROBERTO
CNPJ Business Partners Name: ['JOSE EDVAN ROBERTO']
47 suspect transactions in amount of R$ 130696.80


,issue_date,cnpj_cpf,supplier,total_net_value
229864,2011-01-31T00:00:00,09307232000161,POSTO DE COMBUSTÍVEL SHOPPING LTDA,4500.00
229865,2011-02-28T00:00:00,09307232000161,POSTO DE COMBUSTÍVEL SHOPPING LTDA,4500.00
229866,2011-03-31T00:00:00,09307232000161,POSTO DE COMBUSTÍVEL SHOPPING LTDA,4500.00
229872,2011-04-30T00:00:00,09307232000161,POSTO DE COMBUSTÍVEL SHOPPING LTDA,4500.00
229877,2011-06-30T00:00:00,09307232000161,POSTO DE COMBUSTÍVEL SHOPPING,4500.00
229882,2011-08-01T00:00:00,09307232000161,POSTO DE COMBUSTÍVEL SHOPPING,4500.00
229887,2011-11-16T00:00:00,09307232000161,POSTO DE COMBUSTÍVEIS SHOPPING LTDA,4499.69
229888,2011-12-10T00:00:00,09307232000161,POSTO DE COMBUSTÍVEIS SHOPPING LTDA,4421.59
229919,2013-04-11T00:00:00,09307232000161,POSTO DE COMBUSTÍVEL SHOPPING LTDA,4499.55
229921,2013-07-06T00:00:00,09307232000161,POSTO DE COMBUSTÍVEL SHOPPING LTDA,125.00


## Luiz Fernando Ramos Faria

ATTENTION: In the case bellow we have a fraud! 

Because Luiz Fernando Ramos Faria is brother of Carlos Alberto Ramos de Faria and it's not allowed!

In [85]:
show_fraud_info('LUIZ FERNANDO FARIA', '24574923000115')

Congress Name: LUIZ FERNANDO FARIA
Civil Name: LUIZ FERNANDO RAMOS FARIA
CNPJ Business Partners Name: ['CARLOS ALBERTO RAMOS DE FARIA']
19 suspect transactions in amount of R$ 1916.23


,issue_date,cnpj_cpf,supplier,total_net_value
364090,2011-01-08T00:00:00,24574923000115,POSTO SANTOS DUMONT,105.00
364097,2011-01-25T00:00:00,24574923000115,POSTO SANTOS DUMONT,93.00
364247,2011-08-06T00:00:00,24574923000115,POSTO SANTOS DUMONT,75.00
364249,2011-08-14T00:00:00,24574923000115,POSTO SANTOS DUMONT,140.01
364254,2011-08-20T00:00:00,24574923000115,POSTO SANTOS DUMONT,38.58
364258,2011-08-20T00:00:00,24574923000115,POSTO SANTOS DUMONT,41.42
364304,2011-10-22T00:00:00,24574923000115,POSTO SANTOS DUMONT,81.05
364316,2011-10-28T00:00:00,24574923000115,POSTO STOS.DUMONT,85.01
364330,2011-11-12T00:00:00,24574923000115,POSTO SANTOS DUMONT,50.02
364331,2011-11-13T00:00:00,24574923000115,POSTO SANTOS DUMONT,100.01


# Suspect Nepotism Transactions

These are the cases that I'm not sure yet if they are from the same family or not.

QUESTION: I think that Vitor Penido de Barros is father of Cristiano Sarti Barros but not sure.

We can see something about they here:
http://www.cmnovalima.mg.gov.br/wp-content/uploads/2015/11/Ata-r.-o.-25-03-14.pdf

In [86]:
show_fraud_info('VITOR PENIDO', '00152489000117')

Congress Name: VITOR PENIDO
Civil Name: VITOR PENIDO DE BARROS
CNPJ Business Partners Name: ['CRISTIANO SARTI BARROS']
13 suspect transactions in amount of R$ 43320.99


,issue_date,cnpj_cpf,supplier,total_net_value
441117,2011-01-14T00:00:00,00152489000117,POSTO BICAME LTDA,3600.00
441153,2011-02-25T00:00:00,00152489000117,POSTO BICAME LTDA,2000.00
441181,2011-03-30T00:00:00,00152489000117,POSTO BICAME LTDA.,1666.00
441195,2011-04-29T00:00:00,00152489000117,POSTO BICAME LTDA,3700.00
441231,2011-05-31T00:00:00,00152489000117,POSTO BICAME LTDA,3600.00
441250,2011-06-29T00:00:00,00152489000117,POSTO BICAME LTDA,3400.00
441284,2011-07-29T00:00:00,00152489000117,POSTO BICAME LTDA.,3500.00
441320,2011-08-30T00:00:00,00152489000117,POSTO BICAME LTDA.,3650.00
441322,2011-09-28T00:00:00,00152489000117,POSTO BICAME LTDA,3650.00
441340,2011-10-31T00:00:00,00152489000117,POSTO BICAME LTDA,3650.00


QUESTION: Are Jorge Corte Real and Angela Corte Real de Moraes in the same family?

In [87]:
show_fraud_info('JORGE CÔRTE REAL', '08872798000173')

Congress Name: JORGE CÔRTE REAL
Civil Name: JORGE WICKS CORTE REAL
CNPJ Business Partners Name: ['ANGELA CORTE REAL DE MORAES']
3 suspect transactions in amount of R$ 1217.58


,issue_date,cnpj_cpf,supplier,total_net_value
550910,2011-03-10T00:00:00,08872798000173,AJAX VIAGENS E TURISMO,733.96
550911,2011-02-02T00:00:00,08872798000173,AJAX VIAGENS E TURISMO,328.62
550912,2011-02-03T00:00:00,08872798000173,AJAX VIAGENS E TURISMO,155.00


QUESTION: Are FERNANDO ANTONIO CECILIANO JORDAO and MARCELO ANTONIO LIMA JORDAO in the same family?

In [88]:
show_fraud_info('FERNANDO JORDÃO', '02409350000131')

Congress Name: FERNANDO JORDÃO
Civil Name: FERNANDO ANTONIO CECILIANO JORDAO
CNPJ Business Partners Name: ['MARCELO ANTONIO LIMA JORDAO']
6 suspect transactions in amount of R$ 1043.04


,issue_date,cnpj_cpf,supplier,total_net_value
828133,2012-11-15T00:00:00,02409350000131,AUTO POSTO ESAL LTDA,177.02
828134,2012-11-10T00:00:00,02409350000131,AUTO POSTO ESAL LTDA,180.00
828135,2012-11-08T00:00:00,02409350000131,AUTO POSTO ESAL LTDA,187.00
828167,2013-01-01T00:00:00,02409350000131,auto posto Esal ltda,120.00
828192,2013-02-22T00:00:00,02409350000131,auto posto Esal ltda,186.00
828224,2013-03-22T00:00:00,02409350000131,auto posto Esal ltda,193.02


In [89]:
show_fraud_info('FERNANDO JORDÃO', '03825735000142')

Congress Name: FERNANDO JORDÃO
Civil Name: FERNANDO ANTONIO CECILIANO JORDAO
CNPJ Business Partners Name: ['LUIZ ANTONIO CECILIANO JORDAO', 'MARCELO ANTONIO LIMA JORDAO']
1 suspect transactions in amount of R$ 143.02


,issue_date,cnpj_cpf,supplier,total_net_value
828174,2013-01-31T00:00:00,03825735000142,POSTO DA PRAIA DA RIBEIRA LTDA,143.02


In [90]:
# 773/1089: BETINHO ROSADO - CNPJ: 12997664000156
show_fraud_info('BETINHO ROSADO', '12997664000156')

Congress Name: BETINHO ROSADO
Civil Name: CARLOS ALBERTO DE SOUSA ROSADO
CNPJ Business Partners Name: ['CARLOS JERONIMO DIX SEPT ROSADO MAIA']
24 suspect transactions in amount of R$ 79423.34


,issue_date,cnpj_cpf,supplier,total_net_value
915100,2012-12-07T00:00:00,12997664000156,POSTO LASER LTDA,2720.52
915131,2013-02-27T00:00:00,12997664000156,POSTO LASER LTDA,3220.85
915144,2013-03-26T00:00:00,12997664000156,POSTO LASER LTDA,2810.00
915160,2013-04-23T00:00:00,12997664000156,POSTO LASER LTDA,3120.00
915187,2013-05-27T00:00:00,12997664000156,POSTO LASER LTDA,2719.33
915208,2013-06-24T00:00:00,12997664000156,POSTO LASER LTDA,3210.41
915223,2013-07-26T00:00:00,12997664000156,POSTO LASER LTDA,3120.29
915236,2013-08-28T00:00:00,12997664000156,POSTO LASER LTDA,3481.33
915259,2013-09-30T00:00:00,12997664000156,POSTO LASER LTDA,3250.00
915283,2013-10-30T00:00:00,12997664000156,POSTO LASER LTDA,3375.37


In [91]:
# 773/1089: BETINHO ROSADO - CNPJ: 24186785000105
show_fraud_info('BETINHO ROSADO', '24186785000105')

Congress Name: BETINHO ROSADO
Civil Name: CARLOS ALBERTO DE SOUSA ROSADO
CNPJ Business Partners Name: ['ADALGISA DE SOUSA ROSADO', 'ISAURA AMELIA DE SOUSA ROSADO MAIA', 'CARLOS JERONIMO DIX SEPT ROSADO MAIA']
1 suspect transactions in amount of R$ 155.00


,issue_date,cnpj_cpf,supplier,total_net_value
915378,2014-04-05T00:00:00,24186785000105,POSTO LESTE OESTE,155.0


In [92]:
# 898/1089: RAQUEL MUNIZ - CNPJ: 16902140000195
show_fraud_info('RAQUEL MUNIZ', '16902140000195')

Congress Name: RAQUEL MUNIZ
Civil Name: TANIA RAQUEL DE QUEIROZ MUNIZ
CNPJ Business Partners Name: ['ELZA GONCALVES DE QUEIROZ']
1 suspect transactions in amount of R$ 10.00


,issue_date,cnpj_cpf,supplier,total_net_value
991772,2015-06-30T00:00:00,16902140000195,PAPELARIA XODO LTDA,10.0


In [93]:
# 971/1089: FÁBIO MITIDIERI - CNPJ: 00826182000154
show_fraud_info('FÁBIO MITIDIERI', '00826182000154')

Congress Name: FÁBIO MITIDIERI
Civil Name: FABIO CRUZ MITIDIERI
CNPJ Business Partners Name: ['MAISA CRUZ MITIDIERI']
1 suspect transactions in amount of R$ 415.80


,issue_date,cnpj_cpf,supplier,total_net_value
1039714,2015-08-13T00:00:00,00826182000154,JATOBA HOTEIS E TURISMO LTDA,415.8


In [94]:
# 1034/1089: LUCIO MOSQUINI - CNPJ: 11601349000103
show_fraud_info('LUCIO MOSQUINI', '11601349000103')

Congress Name: LUCIO MOSQUINI
Civil Name: LUCIO ANTONIO MOSQUINI
CNPJ Business Partners Name: ['ANGELA DA COSTA ESTRAL MOSQUINI']
1 suspect transactions in amount of R$ 183.74


,issue_date,cnpj_cpf,supplier,total_net_value
1077922,2015-10-13T00:00:00,11601349000103,E.M COMERCIO DE COMBUSTIVEIS LTDA,183.74


# Business Partners Transactions

These are the curious cases where they spend a lot of money in companies that belongs to their business partners.

In [95]:
show_fraud_info('MARIA HELENA', '15522508000127')

Congress Name: MARIA HELENA
Civil Name: MARIA HELENA VERONESE RODRIGUES
CNPJ Business Partners Name: ['ROGERIO PADILHA KEMPFER', 'HENRIQUE PADILHA KEMPFER']
2 suspect transactions in amount of R$ 72.30


,issue_date,cnpj_cpf,supplier,total_net_value
178832,2015-10-07T00:00:00,15522508000127,VILLE ROY GRILL,37.3
178833,2015-10-14T00:00:00,15522508000127,VILLE ROY GRILL,35.0


In [96]:
show_fraud_info('CIRO NOGUEIRA', '01971900000149')

Congress Name: CIRO NOGUEIRA
Civil Name: CIRO NOGUEIRA LIMA FILHO
CNPJ Business Partners Name: ['JULIO FERRAZ ARCOVERDE']
2 suspect transactions in amount of R$ 9540.00


,issue_date,cnpj_cpf,supplier,total_net_value
68145,2011-01-31T00:00:00,01971900000149,TREVO LOCADORA DE VEÍCULOS LTDA.,3420.0
68146,2011-01-31T00:00:00,01971900000149,TREVO LOCADORA DE VEÍCULOS LTDA,6120.0


In [97]:
show_fraud_info('FELIPE MAIA', '07083712000160')

Congress Name: FELIPE MAIA
Civil Name: FELIPE CATALAO MAIA
CNPJ Business Partners Name: ['MARIA LUCIA DE OLIVEIRA FARIAS']
39 suspect transactions in amount of R$ 252700.00


,issue_date,cnpj_cpf,supplier,total_net_value
314057,2011-06-15T00:00:00,07083712000160,PG PRIME AUTOMÓVEIS LTDA,6000.0
314090,2011-07-01T00:00:00,07083712000160,PG PRIME AUTOMÓVEIS LTDA,6000.0
314117,2011-08-18T00:00:00,07083712000160,PG PRIME AUTOMÓVEIS LTDA,6000.0
314125,2011-09-09T00:00:00,07083712000160,PG PRIME AUTOMÓVEIS LTDA,6000.0
314136,2011-10-05T00:00:00,07083712000160,PG PRIME AUTOMÓVEIS LTDA,6000.0
314174,2011-12-06T00:00:00,07083712000160,PG PRIME AUTOMÓVEIS LTDA,6600.0
314196,2012-01-13T00:00:00,07083712000160,PG PRIME AUTOMÓVEIS LTDA.,6600.0
314212,2012-02-23T00:00:00,07083712000160,PG PRIME AUTOMÓVEIS LTDA,6600.0
314227,2012-03-06T00:00:00,07083712000160,PG PRIME AUTOMÓVEIS LTDA,6600.0
314241,2012-04-03T00:00:00,07083712000160,PG PRIME AUTOMÓVEIS LTDA,6600.0


In [98]:
show_fraud_info('BERINHO BANTIM', '14331631000106')

Congress Name: BERINHO BANTIM
Civil Name: HERBSON JAIRO RIBEIRO BANTIM
CNPJ Business Partners Name: ['WILSON CESAR DE BARROS']
2 suspect transactions in amount of R$ 19000.00


,issue_date,cnpj_cpf,supplier,total_net_value
645464,2011-02-28T00:00:00,14331631000106,TAUÃ PRODUÇÕES,9500.0
645485,2011-03-30T00:00:00,14331631000106,TAUÃ PRODUÇÕES,9500.0


In [99]:
show_fraud_info('JOSÉ NUNES', '08008566000171')

Congress Name: JOSÉ NUNES
Civil Name: JOSE NUNES SOARES
CNPJ Business Partners Name: ['RANULFO DE ABREU CAMPOS']
2 suspect transactions in amount of R$ 200.00


,issue_date,cnpj_cpf,supplier,total_net_value
693422,2014-05-10T00:00:00,08008566000171,POSTO DE COMBUSTIVEL RALENE,100.0
693812,2014-10-03T00:00:00,08008566000171,POSTO DE COMBUSTIVEL RALENE,100.0


In [100]:
# 758/1089: LUCIANO CASTRO - CNPJ: 01271789000188
show_fraud_info('LUCIANO CASTRO', '01271789000188')

Congress Name: LUCIANO CASTRO
Civil Name: LUCIANO DE SOUZA CASTRO
CNPJ Business Partners Name: ['LIONETE MARIA COUTINHO']
3 suspect transactions in amount of R$ 2444.68


,issue_date,cnpj_cpf,supplier,total_net_value
909142,2012-08-28T00:00:00,01271789000188,AIPANA PLAZA HOTEL LTDA,1064.68
909376,2013-02-24T00:00:00,01271789000188,AIPANA PLAZA HOTEL LTDA,480.00
909492,2013-06-23T00:00:00,01271789000188,AIPANA PLAZA HOTEL LTDA,900.00


In [101]:
# 770/1089: ARMANDO ABÍLIO - CNPJ: 12610937000168
show_fraud_info('ARMANDO ABÍLIO', '12610937000168')

Congress Name: ARMANDO ABÍLIO
Civil Name: ARMANDO ABILIO VIEIRA
CNPJ Business Partners Name: ['MAYSA AYRES DA MOTTA BENEVIDES GADELHA', 'DALTON ROBERTO BENEVIDES GADELHA', 'FELIPE MOTTA BENEVIDES GADELHA']
2 suspect transactions in amount of R$ 6000.00


,issue_date,cnpj_cpf,supplier,total_net_value
913641,2012-02-29T00:00:00,12610937000168,RÁDIO CIDADE ESPERANÇA LTDA.,3000.0
913646,2012-03-28T00:00:00,12610937000168,RÁDIO CIDADE ESPERANÇA LTDA,3000.0


In [102]:
# 934/1089: MARINALDO ROSENDO - CNPJ: 10651703000133
show_fraud_info('MARINALDO ROSENDO', '10651703000133')

Congress Name: MARINALDO ROSENDO
Civil Name: MARINALDO ROSENDO DE ALBUQUERQUE
CNPJ Business Partners Name: ['SANDRO LUIZ GUEDES BARBOSA']
1 suspect transactions in amount of R$ 95.02


,issue_date,cnpj_cpf,supplier,total_net_value
1015080,2015-08-21T00:00:00,10651703000133,AUTO POSTO BARBOSA LTDA,95.02


In [103]:
# 977/1089: RODRIGO PACHECO - CNPJ: 17272550000162
show_fraud_info('RODRIGO PACHECO', '17272550000162')

Congress Name: RODRIGO PACHECO
Civil Name: RODRIGO OTAVIO SOARES PACHECO
CNPJ Business Partners Name: ['LEONARDO LEMOS SILVEIRA']
1 suspect transactions in amount of R$ 578.66


,issue_date,cnpj_cpf,supplier,total_net_value
1043592,2015-08-06T00:00:00,17272550000162,POSTO AEL SÃO JOSÉ LTDA,578.66


# Conclusion

Besides the database is hard to collect, because I spent a lot of time collecting information from ConsultaSocio and CNPJ database this system is really good to identify suspect transactions.

The idea here is to identify family that has business together and check how they spend our money there.

Now we have something about R$ 176.613,03 that we know is money spent in illegal way.

We suspect that R$ 125.912,51 is spent in family business but needs further investigation.

We also have R$ 290.630,66 that is spent in business partners companies and maybe we can have some conflict of interests when contracting this companies.